In [6]:
import sys, os #imp
sys.path.append('..')
import pandas as pd
import yaml
import backtesting as bt
import matplotlib.pyplot as plt

from tools.instruments import instruments



In [10]:
%load_ext autoreload
%aimport backtesting.system
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plt.ioff()

In [3]:
with open('systems/sample.yaml', encoding='utf-8') as f:
    system1 = yaml.load(f, Loader=yaml.FullLoader)
    print(system1)

{'name': 'testSystem', 'description': '시스템 설명', 'sectors': 'default', 'instruments': ['AD'], 'from_date': '2019-08-01', 'to_date': None, 'principal': 1000000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.3, 'max_sector_heat': 0.3, 'max_trade_heat': 0.01, 'max_lots': 100, 'commission': 5.98, 'skid': 0.25, 'metrics': [['ema30', 'EMA', 'window=30'], ['ema5', 'EMA', 'window=5'], ['min30', 'MIN', 'window=30'], ['max300', 'MAX', 'window=300']], 'entry_rule': {'long': 'ema5>ema30', 'short': None}, 'exit_rule': {'long': 'ema5<ema30', 'short': None}, 'stop_rule': {'long': 'min30', 'short': None}}


In [4]:
trader = bt.Trader(systems=[system1])
system = trader.systems[0]
trader.run()

매매시작
거래일: 2019-08-01 00:00:00, 시스템: testSystem 3.6023647785186768sec
거래일: 2019-08-02 00:00:00, 시스템: testSystem 0.001998424530029297sec
거래일: 2019-08-05 00:00:00, 시스템: testSystem 0.0019626617431640625sec
거래일: 2019-08-06 00:00:00, 시스템: testSystem 0.001995563507080078sec
거래일: 2019-08-07 00:00:00, 시스템: testSystem 0.0019943714141845703sec
거래일: 2019-08-08 00:00:00, 시스템: testSystem 0.0019943714141845703sec
거래일: 2019-08-09 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2019-08-12 00:00:00, 시스템: testSystem 0.0009975433349609375sec
거래일: 2019-08-13 00:00:00, 시스템: testSystem 0.0015063285827636719sec
거래일: 2019-08-14 00:00:00, 시스템: testSystem 0.0019941329956054688sec
거래일: 2019-08-15 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2019-08-16 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2019-08-19 00:00:00, 시스템: testSystem 0.0009970664978027344sec
거래일: 2019-08-20 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2019-08-21 00:00:00, 시스템: testSystem 0.001994609832763672s

거래일: 2020-03-05 00:00:00, 시스템: testSystem 0.001993417739868164sec
거래일: 2020-03-06 00:00:00, 시스템: testSystem 0.0019953250885009766sec
거래일: 2020-03-09 00:00:00, 시스템: testSystem 0.0019941329956054688sec
거래일: 2020-03-10 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2020-03-11 00:00:00, 시스템: testSystem 0.0020036697387695312sec
거래일: 2020-03-12 00:00:00, 시스템: testSystem 0.001985311508178711sec
거래일: 2020-03-13 00:00:00, 시스템: testSystem 0.001995086669921875sec
거래일: 2020-03-16 00:00:00, 시스템: testSystem 0.0019943714141845703sec
거래일: 2020-03-17 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2020-03-18 00:00:00, 시스템: testSystem 0.001007080078125sec
거래일: 2020-03-19 00:00:00, 시스템: testSystem 0.001984834671020508sec
거래일: 2020-03-20 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2020-03-23 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2020-03-24 00:00:00, 시스템: testSystem 0.000997304916381836sec
거래일: 2020-03-25 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 20

거래일: 2020-09-01 00:00:00, 시스템: testSystem 0.04787158966064453sec
거래일: 2020-09-02 00:00:00, 시스템: testSystem 0.047872066497802734sec
거래일: 2020-09-03 00:00:00, 시스템: testSystem 0.044881343841552734sec
거래일: 2020-09-04 00:00:00, 시스템: testSystem 0.04591178894042969sec
거래일: 2020-09-07 00:00:00, 시스템: testSystem 0.044872283935546875sec
거래일: 2020-09-08 00:00:00, 시스템: testSystem 0.0019941329956054688sec
거래일: 2020-09-09 00:00:00, 시스템: testSystem 0.0438847541809082sec
거래일: 2020-09-10 00:00:00, 시스템: testSystem 0.0468447208404541sec
거래일: 2020-09-11 00:00:00, 시스템: testSystem 0.047872066497802734sec
거래일: 2020-09-14 00:00:00, 시스템: testSystem 0.046874284744262695sec
거래일: 2020-09-15 00:00:00, 시스템: testSystem 0.04587721824645996sec
거래일: 2020-09-16 00:00:00, 시스템: testSystem 0.04488182067871094sec
거래일: 2020-09-17 00:00:00, 시스템: testSystem 0.04587721824645996sec
거래일: 2020-09-18 00:00:00, 시스템: testSystem 0.04687309265136719sec
거래일: 2020-09-21 00:00:00, 시스템: testSystem 0.0468745231628418sec
거래일: 2020-09-22 00:00

거래일: 2021-04-26 00:00:00, 시스템: testSystem 0.00695347785949707sec
거래일: 2021-04-27 00:00:00, 시스템: testSystem 0.008975982666015625sec
거래일: 2021-04-28 00:00:00, 시스템: testSystem 0.007978439331054688sec
거래일: 2021-04-29 00:00:00, 시스템: testSystem 0.008976221084594727sec
거래일: 2021-04-30 00:00:00, 시스템: testSystem 0.008975982666015625sec
거래일: 2021-05-03 00:00:00, 시스템: testSystem 0.009972810745239258sec
거래일: 2021-05-04 00:00:00, 시스템: testSystem 0.0109710693359375sec
거래일: 2021-05-05 00:00:00, 시스템: testSystem 0.011967897415161133sec
거래일: 2021-05-06 00:00:00, 시스템: testSystem 0.011968374252319336sec
거래일: 2021-05-07 00:00:00, 시스템: testSystem 0.012965202331542969sec
거래일: 2021-05-10 00:00:00, 시스템: testSystem 0.013962507247924805sec
거래일: 2021-05-11 00:00:00, 시스템: testSystem 0.016954660415649414sec
거래일: 2021-05-12 00:00:00, 시스템: testSystem 0.014960050582885742sec
거래일: 2021-05-13 00:00:00, 시스템: testSystem 0.015957355499267578sec
거래일: 2021-05-14 00:00:00, 시스템: testSystem 0.016954660415649414sec
거래일: 2021-05-

거래일: 2021-11-02 00:00:00, 시스템: testSystem 0.012964963912963867sec
거래일: 2021-11-03 00:00:00, 시스템: testSystem 0.013962030410766602sec
거래일: 2021-11-04 00:00:00, 시스템: testSystem 0.013962984085083008sec
거래일: 2021-11-05 00:00:00, 시스템: testSystem 0.01595759391784668sec
거래일: 2021-11-08 00:00:00, 시스템: testSystem 0.016954421997070312sec
거래일: 2021-11-09 00:00:00, 시스템: testSystem 0.01795196533203125sec
거래일: 2021-11-10 00:00:00, 시스템: testSystem 0.01795196533203125sec
거래일: 2021-11-11 00:00:00, 시스템: testSystem 0.008975982666015625sec
거래일: 2021-11-12 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2021-11-15 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2021-11-16 00:00:00, 시스템: testSystem 0.0019948482513427734sec
거래일: 2021-11-17 00:00:00, 시스템: testSystem 0.0010209083557128906sec
거래일: 2021-11-18 00:00:00, 시스템: testSystem 0.0019941329956054688sec
거래일: 2021-11-19 00:00:00, 시스템: testSystem 0.001994609832763672sec
거래일: 2021-11-22 00:00:00, 시스템: testSystem 0.001995563507080078sec
거래일: 2021-

In [13]:
system.create_report()

{'name': 'testSystem', 'description': '시스템 설명', 'sectors': 'default', 'instruments': ['AD'], 'from_date': '2019-08-01', 'to_date': None, 'principal': 1000000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.3, 'max_sector_heat': 0.3, 'max_trade_heat': 0.01, 'max_lots': 100, 'commission': 5.98, 'skid': 0.25, 'metrics': [['ema30', 'EMA', 'window=30'], ['ema5', 'EMA', 'window=5'], ['min30', 'MIN', 'window=30'], ['max300', 'MAX', 'window=300']], 'entry_rule': {'long': 'ema5>ema30', 'short': None}, 'exit_rule': {'long': 'ema5<ema30', 'short': None}, 'stop_rule': {'long': 'min30', 'short': None}}
ema5<ema30
<html>

<body align="center">
    <div>
        <h1> 시스템 성능 보고서 </h1>
        <h3> 테스트 날짜: 2023년 06월 25일</h3>
        <div align="center">
            <table border="1px solid black" width="800px" style="border-collapse:collapse">
                <tr>
                    <th width="20%"> 시스템명</th>
                    <td style="padding-left:10px"> testSystem</td>
                </tr>


In [18]:
trader.quotes.columns

MultiIndex([('AD',   'open'),
            ('AD',   'high'),
            ('AD',    'low'),
            ('AD',  'close'),
            ('AD', 'volume'),
            ('AD',     'oi'),
            ('BO',   'open'),
            ('BO',   'high'),
            ('BO',    'low'),
            ('BO',  'close'),
            ...
            ('US',    'low'),
            ('US',  'close'),
            ('US', 'volume'),
            ('US',     'oi'),
            ( 'W',   'open'),
            ( 'W',   'high'),
            ( 'W',    'low'),
            ( 'W',  'close'),
            ( 'W', 'volume'),
            ( 'W',     'oi')],
           names=['symbol', 'field'], length=234)

In [21]:
trader.quotes.columns.levels[0].to_list()

['AD',
 'BO',
 'BP',
 'C',
 'CC',
 'CD',
 'CL',
 'CT',
 'DX',
 'EC',
 'ES',
 'FV',
 'GC',
 'HG',
 'HO',
 'JY',
 'KC',
 'LC',
 'LN',
 'MD',
 'NE',
 'NG',
 'NK',
 'NQ',
 'O',
 'OJ',
 'PA',
 'PL',
 'RB',
 'RTY',
 'S',
 'SB',
 'SF',
 'SI',
 'SM',
 'TU',
 'TY',
 'US',
 'W']